In [1]:
# Importamos las librerías que necesitamos

# Librerías de extracción de datos
# -----------------------------------------------------------------------

# Importaciones:
# Beautifulsoup
from bs4 import BeautifulSoup

# Requests
import requests

import pandas as pd
import numpy as np

from tqdm import tqdm
from time import sleep

from geopy.geocoders import Nominatim

# Importar librerías para automatización de navegadores web con Selenium
# -----------------------------------------------------------------------
from selenium import webdriver  # Selenium es una herramienta para automatizar la interacción con navegadores web.
from webdriver_manager.chrome import ChromeDriverManager  # ChromeDriverManager gestiona la instalación del controlador de Chrome.
from selenium.webdriver.common.keys import Keys  # Keys es útil para simular eventos de teclado en Selenium.
from selenium.webdriver.support.ui import Select  # Select se utiliza para interactuar con elementos <select> en páginas web.
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException # Excepciones comunes de selenium que nos podemos encontrar 


In [2]:
url_tiempo = 'https://www.wunderground.com/history/monthly/es/madrid/LEMD/date/2024-1'

response_tiempo = requests.get(url_tiempo)
print(response_tiempo.status_code)

sopa_tiempo = BeautifulSoup(response_tiempo.content, 'html.parser')

200


In [3]:
# Abre Chrome
driver = webdriver.Chrome()

# Definimos una URL
url_wunder = "https://www.wunderground.com/history/monthly/es/madrid/LEMD/date/2024-1"

# Abrir la URL
driver.get(url_wunder)
# Maximizar ventana
driver.maximize_window()

print('Hemos cargado la página en el navegador')

driver.implicitly_wait(20)
iframe = WebDriverWait(driver, 10).until(EC.presence_of_element_located(('xpath', '//*[@id="sp_message_iframe_1165301"]')))
driver.switch_to.frame(iframe)

try:
    driver.find_element('css selector', '#notice > div.message-component.message-row.cta-buttons-container > div.message-component.message-column.cta-button-column.reject-column > button').click()
    print('Hemos quitado cookies')
    
except:
    print('No encuentro el botón')

Hemos cargado la página en el navegador
Hemos quitado cookies


In [4]:
#display table on the page with scroll
#sleep(3)
#driver.execute_script('window.scrollBy(0, 2000)')
sleep(3)
html_table_page = driver.page_source
bs_table = BeautifulSoup(html_table_page)

In [29]:
tabla = bs_table.findAll('table', {'aria-labelledby': "History days"})

In [39]:
print(tabla[0])

<table _ngcontent-app-root-c196="" aria-labelledby="History days" class="days ng-star-inserted"><thead _ngcontent-app-root-c196=""><tr _ngcontent-app-root-c196=""><td _ngcontent-app-root-c196="" class="ng-star-inserted">Time</td><td _ngcontent-app-root-c196="" class="ng-star-inserted">Temperature (°F)</td><td _ngcontent-app-root-c196="" class="ng-star-inserted">Dew Point (°F)</td><td _ngcontent-app-root-c196="" class="ng-star-inserted">Humidity (%)</td><td _ngcontent-app-root-c196="" class="ng-star-inserted">Wind Speed (mph)</td><td _ngcontent-app-root-c196="" class="ng-star-inserted">Pressure (in)</td><td _ngcontent-app-root-c196="" class="ng-star-inserted">Precipitation (in)</td><!-- --></tr></thead><tbody _ngcontent-app-root-c196=""><tr _ngcontent-app-root-c196=""><td _ngcontent-app-root-c196="" class="ng-star-inserted"><table _ngcontent-app-root-c196="" aria-labelledby="Days data"><tr _ngcontent-app-root-c196="" class="ng-star-inserted"><td _ngcontent-app-root-c196="" class="ng-sta

In [50]:
# Encabezados
tabla[0].find('tr').findAll('td')

[<td _ngcontent-app-root-c196="" class="ng-star-inserted">Time</td>,
 <td _ngcontent-app-root-c196="" class="ng-star-inserted">Temperature (°F)</td>,
 <td _ngcontent-app-root-c196="" class="ng-star-inserted">Dew Point (°F)</td>,
 <td _ngcontent-app-root-c196="" class="ng-star-inserted">Humidity (%)</td>,
 <td _ngcontent-app-root-c196="" class="ng-star-inserted">Wind Speed (mph)</td>,
 <td _ngcontent-app-root-c196="" class="ng-star-inserted">Pressure (in)</td>,
 <td _ngcontent-app-root-c196="" class="ng-star-inserted">Precipitation (in)</td>]

In [148]:
tabla[0].find('tbody').text

' Jan  1  2  3  4  5  6  7  8  9  10  11  12  13  14  15  16  17  18  19  20  21  22  23  24  25  26  27  28  29  30  31  Max  Avg  Min  50  41.0  37  45  40.3  34  55  50.4  43  50  48.0  46  50  45.7  41  50  40.9  34  50  38.2  30  46  36.3  27  46  41.1  37  48  43.5  39  48  39.4  32  46  35.0  27  48  38.3  30  46  43.2  39  59  49.2  43  59  56.1  52  55  53.5  50  59  54.0  52  52  44.7  39  50  42.5  34  54  44.8  36  48  39.6  32  61  44.0  32  68  48.5  36  66  48.2  36  68  49.1  37  68  49.5  37  64  48.1  36  61  48.7  39  57  45.6  37  50  40.6  34  Max  Avg  Min  41  37.4  34  43  38.3  32  54  48.9  43  50  47.5  46  46  37.5  32  34  31.5  28  34  29.3  27  32  28.5  25  37  35.6  32  41  39.1  37  39  32.5  27  32  27.6  23  41  33.5  27  45  41.3  36  55  48.1  41  55  51.5  45  50  48.0  46  50  47.9  46  50  42.3  32  43  35.6  30  41  38.4  34  43  36.9  32  50  40.8  32  52  43.7  36  52  42.8  36  54  43.1  36  50  42.6  36  46  41.7  36  45  41.9  37  46  41.5

In [136]:
lista_valores_tabla = tabla[0].find('tbody').text.split(r'\t')

In [137]:
print(lista_valores_tabla)

[' Jan  1  2  3  4  5  6  7  8  9  10  11  12  13  14  15  16  17  18  19  20  21  22  23  24  25  26  27  28  29  30  31  Max  Avg  Min  50  41.0  37  45  40.3  34  55  50.4  43  50  48.0  46  50  45.7  41  50  40.9  34  50  38.2  30  46  36.3  27  46  41.1  37  48  43.5  39  48  39.4  32  46  35.0  27  48  38.3  30  46  43.2  39  59  49.2  43  59  56.1  52  55  53.5  50  59  54.0  52  52  44.7  39  50  42.5  34  54  44.8  36  48  39.6  32  61  44.0  32  68  48.5  36  66  48.2  36  68  49.1  37  68  49.5  37  64  48.1  36  61  48.7  39  57  45.6  37  50  40.6  34  Max  Avg  Min  41  37.4  34  43  38.3  32  54  48.9  43  50  47.5  46  46  37.5  32  34  31.5  28  34  29.3  27  32  28.5  25  37  35.6  32  41  39.1  37  39  32.5  27  32  27.6  23  41  33.5  27  45  41.3  36  55  48.1  41  55  51.5  45  50  48.0  46  50  47.9  46  50  42.3  32  43  35.6  30  41  38.4  34  43  36.9  32  50  40.8  32  52  43.7  36  52  42.8  36  54  43.1  36  50  42.6  36  46  41.7  36  45  41.9  37  46  41.

In [140]:
mes = lista_valores_tabla[0]
dias = lista_valores_tabla[0:32]
max_val = lista_valores_tabla[32:541:3]
avg_val = lista_valores_tabla[33:542:3]
min_val = lista_valores_tabla[34:543:3]

"""
df = pd.DataFrame({
    'Día': dias,
    'Max': max_val,
    'Avg': avg_val,
    'Min': min_val
})
"""

"\ndf = pd.DataFrame({\n    'Día': dias,\n    'Max': max_val,\n    'Avg': avg_val,\n    'Min': min_val\n})\n"

In [139]:
mes = lista_valores_tabla[0]
dias = lista_valores_tabla[0:32]
max_val = lista_valores_tabla[32:(32+32*3):3]
avg_val = lista_valores_tabla[33:(32+32*3):3]
min_val = lista_valores_tabla[34:(32+32*3):3]

In [147]:
tabla[0]

<table _ngcontent-app-root-c196="" aria-labelledby="History days" class="days ng-star-inserted"><thead _ngcontent-app-root-c196=""><tr _ngcontent-app-root-c196=""><td _ngcontent-app-root-c196="" class="ng-star-inserted">Time</td><td _ngcontent-app-root-c196="" class="ng-star-inserted">Temperature (°F)</td><td _ngcontent-app-root-c196="" class="ng-star-inserted">Dew Point (°F)</td><td _ngcontent-app-root-c196="" class="ng-star-inserted">Humidity (%)</td><td _ngcontent-app-root-c196="" class="ng-star-inserted">Wind Speed (mph)</td><td _ngcontent-app-root-c196="" class="ng-star-inserted">Pressure (in)</td><td _ngcontent-app-root-c196="" class="ng-star-inserted">Precipitation (in)</td><!-- --></tr></thead><tbody _ngcontent-app-root-c196=""><tr _ngcontent-app-root-c196=""><td _ngcontent-app-root-c196="" class="ng-star-inserted"><table _ngcontent-app-root-c196="" aria-labelledby="Days data"><tr _ngcontent-app-root-c196="" class="ng-star-inserted"><td _ngcontent-app-root-c196="" class="ng-sta

In [164]:
# Mirar esto!!!!!

tabla[0].findAll('table', {'aria-labelledby': 'Days data'})[1].findChildren('tr', recursive =False)[0].text


' Max  Avg  Min '

In [146]:
tabla[0].findAll('table')[1].text

' Max  Avg  Min  50  41.0  37  45  40.3  34  55  50.4  43  50  48.0  46  50  45.7  41  50  40.9  34  50  38.2  30  46  36.3  27  46  41.1  37  48  43.5  39  48  39.4  32  46  35.0  27  48  38.3  30  46  43.2  39  59  49.2  43  59  56.1  52  55  53.5  50  59  54.0  52  52  44.7  39  50  42.5  34  54  44.8  36  48  39.6  32  61  44.0  32  68  48.5  36  66  48.2  36  68  49.1  37  68  49.5  37  64  48.1  36  61  48.7  39  57  45.6  37  50  40.6  34 '

### Geopy

In [7]:
"""lista_municipios = ['acebeda-la', 'ajalvir', 'alameda-del-valle', 'alamo-el', 'alcala-de-henares', 'alcobendas', 'alcorcon', 'aldea-del-fresno', 'algete', 'alpedrete', 'ambite', 'anchuelo', 'aranjuez', 'arganda-del-rey', 'arroyomolinos', 'atazar-el', 'batres', 'becerril-de-la-sierra', 'belmonte-de-tajo', 'berrueco-el', 'berzosa-del-lozoya', 'boadilla-del-monte', 'boalo-el', 'braojos', 'brea-de-tajo', 'brunete', 'buitrago-del-lozoya', 'bustarviejo', 'cabanillas-de-la-sierra', 'cabrera-la', 'cadalso-de-los-vidrios', 'camarma-de-esteruelas', 'campo-real', 'canencia', 'carabana', 'casarrubuelos', 'cenicientos', 'cercedilla', 'cervera-de-buitrago', 'chapineria', 'chinchon', 'ciempozuelos', 'cobena', 'collado-mediano', 'collado-villalba', 'colmenar-del-arroyo', 'colmenar-de-oreja', 'colmenarejo', 'colmenar-viejo', 'corpa', 'coslada', 'cubas-de-la-sagra', 'daganzo-de-arriba', 'escorial-el', 'estremera', 'fresnedillas-de-la-oliva', 'fresno-de-torote', 'fuenlabrada', 'fuente-el-saz-de-jarama', 'fuentiduena-de-tajo', 'galapagar', 'garganta-de-los-montes', 'gargantilla-del-lozoya-y-pinilla-de-buitrago', 'gascones', 'getafe', 'grinon', 'guadalix-de-la-sierra', 'guadarrama', 'hiruela-la', 'horcajo-de-la-sierra-aoslos', 'horcajuelo-de-la-sierra', 'hoyo-de-manzanares', 'humanes-de-madrid', 'leganes', 'loeches', 'lozoya', 'lozoyuela-navas-sieteiglesias', 'madarcos', 'madrid', 'majadahonda', 'manzanares-el-real', 'meco', 'mejorada-del-campo', 'miraflores-de-la-sierra', 'molar-el', 'molinos-los', 'montejo-de-la-sierra', 'moraleja-de-enmedio', 'moralzarzal', 'morata-de-tajuna', 'mostoles', 'navacerrada', 'navalafuente', 'navalagamella', 'navalcarnero', 'navarredonda-y-san-mames', 'navas-del-rey', 'nuevo-baztan', 'olmeda-de-las-fuentes', 'orusco-de-tajuna', 'paracuellos-de-jarama', 'parla', 'patones', 'pedrezuela', 'pelayos-de-la-presa', 'perales-de-tajuna', 'pezuela-de-las-torres', 'pinilla-del-valle', 'pinto', 'pinuecar-gandullas', 'pozuelo-de-alarcon', 'pozuelo-del-rey', 'pradena-del-rincon', 'puebla-de-la-sierra', 'puentes-viejas-manjiron', 'quijorna', 'rascafria', 'reduena', 'ribatejada', 'rivas-vaciamadrid', 'robledillo-de-la-jara', 'robledo-de-chavela', 'robregordo', 'rozas-de-madrid-las', 'rozas-de-puerto-real', 'san-agustin-del-guadalix', 'san-fernando-de-henares', 'san-lorenzo-de-el-escorial', 'san-martin-de-la-vega', 'san-martin-de-valdeiglesias', 'san-sebastian-de-los-reyes', 'santa-maria-de-la-alameda', 'santorcaz', 'santos-de-la-humosa-los', 'serna-del-monte-la', 'serranillos-del-valle', 'sevilla-la-nueva', 'somosierra', 'soto-del-real', 'talamanca-de-jarama', 'tielmes', 'titulcia', 'torrejon-de-ardoz', 'torrejon-de-la-calzada', 'torrejon-de-velasco', 'torrelaguna', 'torrelodones', 'torremocha-de-jarama', 'torres-de-la-alameda', 'tres-cantos', 'valdaracete', 'valdeavero', 'valdelaguna', 'valdemanco', 'valdemaqueda', 'valdemorillo', 'valdemoro', 'valdeolmos-alalpardo', 'valdepielagos', 'valdetorres-de-jarama', 'valdilecha', 'valverde-de-alcala', 'velilla-de-san-antonio', 'vellon-el', 'venturada', 'villaconejos', 'villa-del-prado', 'villalbilla', 'villamanrique-de-tajo', 'villamanta', 'villamantilla', 'villanueva-de-la-canada', 'villanueva-del-pardillo', 'villanueva-de-perales', 'villar-del-olmo', 'villarejo-de-salvanes', 'villaviciosa-de-odon', 'villavieja-del-lozoya', 'zarzalejo']

geolocator = Nominatim(user_agent="my_app")
locations = []


location = geolocator.geocode(lista_municipios[4])"""